In [1]:
import pandas as pd
import numpy as np
import lexas.gene_to_sym
gene_to_sym = lexas.gene_to_sym.get()

In [2]:
def newdic():
    return {g:set() for g in gene_to_sym.values()}
def newdic_num():
    return {g:[] for g in gene_to_sym.values()}

# Categorical features

In [3]:
categorical = {}

In [5]:
#chromosome
categorical["Chromosome"] = newdic()
df = pd.read_table("./data/HGNC.txt")
for c,s in zip(df["Chromosome"],df["Approved symbol"]):
    if str(s).upper() in gene_to_sym:
        symbol = gene_to_sym[str(s).upper()]
        c = str(c)
        if "p" in c:
            c = c[:c.index("p")+1]
        elif "q" in c:
            c = c[:c.index("q")+1]
        else:
            continue
        if len(c) > 3 or c=="unp":
            continue
        categorical["Chromosome"][symbol].add(c)

In [225]:
#Gene Ontology

#!wget http://geneontology.org/gene-associations/goa_human.gaf.gz -O Repository/Feature/goa_human.gaf.gz
#!gunzip Repository/Feature/goa_human.gaf.gz
categorical["GO"] =newdic()
dfgo = pd.read_table("./Repository/Feature/goa_human.gaf",skiprows=41,header=None)
for gene,idx in zip(dfgo[2],dfgo[4]):
    gene = str(gene).upper()
    if gene in gene_to_sym:
        symbol = gene_to_sym[gene]
        categorical["GO"][symbol].add(idx)

In [313]:
#MGI
#!wget http://www.informatics.jax.org/downloads/reports/MGI_PhenoGenoMP.rpt -O Repository/Feature/MGI_PhenoGenoMP.rpt
#!wget http://www.informatics.jax.org/downloads/reports/HMD_HumanPhenotype.rpt -O Repository/Feature/HMD_HumanPhenotype.rpt
#!wget http://www.informatics.jax.org/downloads/reports/VOC_MammalianPhenotype.rpt -O Repository/Feature/VOC_MammalianPhenotype.rpt
mousep={}
mgiset = set()
with open("./Repository/Feature/MGI_PhenoGenoMP.rpt","r") as f:
    for line in f:
        ls = line.strip("\n").split("\t")
        if "<" in ls[1]:
            g,p = ls[1][:ls[1].index("<")],ls[3]
            if g.upper() not in mousep:
                mousep[g.upper()] = set()
            mousep[g.upper()].add(p)
            
categorical["MGI"] = newdic()
with open("./Repository/Feature/HMD_HumanPhenotype.rpt","r") as f:
    for line in f:
        ls = line.strip("\n").split("\t")
        h,m=ls[0].upper(),ls[2].upper()
        if m in mousep and h.upper() in gene_to_sym:
            symbol = gene_to_sym[h.upper()]
            categorical["MGI"][symbol] = mousep[m]

In [227]:
#HPO,OMIM,ORPHAN
#!wget http://purl.obolibrary.org/obo/hp/hpoa/genes_to_phenotype.txt -O Repository/Feature/genes_to_phenotype.txt
df_hpo = pd.read_table("./Repository/Feature/genes_to_phenotype.txt",skiprows=1,header=None)
categorical["HPO"] = newdic()
categorical["OMIM"] =  newdic()

for g,h,o in zip(df_hpo[1],df_hpo[2],df_hpo[8]):
    g = str(g).upper()
    if g in gene_to_sym:
        symbol = gene_to_sym[g]
        categorical["HPO"][symbol].add(h)
        categorical["OMIM"][symbol].add(o)

In [228]:
#Transcript factors
#!wget https://maayanlab.cloud/static/hdfs/harmonizome/data/encodetfppi/gene_attribute_matrix.txt.gz -O Repository/Feature/gene_attribute_matrix.txt.gz
#!gunzip Repository/Feature/gene_attribute_matrix.txt.gz

import tqdm.notebook
import collections
dftf = pd.read_table('./Repository/Feature/gene_attribute_matrix.txt',index_col="#")
dftf=dftf.drop(["#.1","GeneSym"],axis=1)
dftf=dftf.drop(["#","GeneSym"],axis=0)
dftf = dftf.astype(float)
dftf.index = [i.upper() for i in dftf.index]
c = collections.Counter(dftf.index)
categorical["TF"]= newdic()

for tf in tqdm.notebook.tqdm(dftf.columns):
    dftf2 = dftf[dftf[tf]==1][tf]
    for g in dftf2.index:
        if g in gene_to_sym and c[g]==1:
            symbol = gene_to_sym[g]
            categorical["TF"][symbol].add("TF: "+tf)

/home/user/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


  0%|          | 0/181 [00:00<?, ?it/s]

In [229]:
#iRefindex

#!wget https://irefindex.vib.be/download/irefindex/data/archive/release_19.0/psi_mitab/MITAB2.6/9606.mitab.08-22-2022.txt.zip \
#    -O ./Repository/Feature/ifrefindex.txt.zip
#!unzip ./Repository/Feature/ifrefindex.txt.zip

f = open("./Repository/Feature/-","r")
categorical["iRefIndex"] = newdic()
c = 0
for line in tqdm.notebook.tqdm(f):
  ls = line.strip("\n").split("\t")
  if ls[4].startswith("hgnc") and ls[5].startswith("hgnc"):
    proa = ls[4].split("|")[0][5:].upper()
    prob = ls[5].split("|")[0][5:].upper()
    if proa in gene_to_sym and prob in gene_to_sym:
        symbol_a = gene_to_sym[proa]
        symbol_b = gene_to_sym[prob]
        categorical["iRefIndex"][symbol_a].add("iRefIndex: "+symbol_b)
        categorical["iRefIndex"][symbol_b].add("iRefIndex: "+symbol_a) 
f.close()

0it [00:00, ?it/s]

In [230]:
#subcellular localization
#!wget https://www.proteinatlas.org/download/subcellular_location.tsv.zip -O ./Repository/Feature/subcellular_location.tsv.zip
#!unzip ./Repository/Feature/subcellular_location.tsv.zip -d ./Repository/Feature
import pandas as pd
categorical["Localization"] = newdic()
df = pd.read_table('./Repository/Feature/subcellular_location.tsv')
for loc,gene in zip(df["Main location"],df["Gene name"]):
    gene = gene.upper()
    if gene in gene_to_sym:
        symbol = gene_to_sym[gene]
        if type(loc)==str:
            categorical["Localization"][symbol] = slodic[symbol] | set(loc.strip().split(";"))
for symbol in slodic:
    categorical["Localization"][symbol] = {"Localization: "+l for l in categorical["Localization"][symbol]}

In [231]:
#DepMap_WebSter
#!wget https://www.cell.com/cms/10.1016/j.cels.2021.12.005/attachment/dbe94bef-7b50-43ea-bc46-415415a813d9/mmc4.xlsx
#%mv mmc4.xlsx ./Repository/Feature/webster.xlsx
dfweb = pd.read_excel("./Repository/Feature/webster.xlsx",sheet_name="Gene meta",engine="openpyxl")
dfweb2 = pd.read_excel("./Repository/Feature/webster.xlsx",sheet_name="Function Names",engine="openpyxl")
function_name = {}
for m,v in zip(dfweb2["Manual Name"],dfweb2["Name"]):
    function_name[v] = m

categorical["WebSter"] = newdic()
symbols = dfweb["symbol"].tolist()
funcs = [dfweb["Function_{}".format(n)].tolist() for n in range(1,5)]
loadings = [dfweb["Loading_{}".format(n)].tolist() for n in range(1,5)]
for n in range(len(dfweb)):
    gene = str(symbols[n])
    if gene.upper() in gene_to_sym:
        symbol = gene_to_sym[gene.upper()]
        for k in range(4):
            if abs(loadings[k][n])>0.1:
                v = funcs[k][n]
                categorical["WebSter"][symbol].add("WebSter: {} ({})".format(v,function_name[v]))

# Numerical features

In [233]:
numerical = {}

In [234]:
#Expression@Tissue
#!wget https://www.proteinatlas.org/download/rna_tissue_hpa.tsv.zip -O Repository/Feature/rna_tissue_hpa.tsv.zip
#!unzip ./Repository/Feature/rna_tissue_hpa.tsv.zip -d ./Repository/Feature
numerical["Tissue_expression"] = newdic_num()
with open("./Repository/Feature/rna_tissue_hpa.tsv","r") as f:
    for line in f:
        ls = line.strip("\n").split("\t")
        if ls[1].upper() in gene_to_sym:
            symbol = gene_to_sym[ls[1].upper()]
            numerical["Tissue_expression"][symbol].append(float(ls[3]))

In [235]:
#Expression@Cancer_cell_lines
#!wget https://cog.sanger.ac.uk/cmp/download/rnaseq_all_20220624.zip -O Repository/Feature/rnaseq_all_20220624.zip
#!unzip ./Repository/Feature/rnaseq_all_20220624.zip -d ./Repository/Feature
numerical["Cancer_expression"] = newdic_num()
with open("./Repository/Feature/rnaseq_tpm_20220624.csv","r") as f:
    for line in f:
        ls = line.strip("\n").split(",")
        if ls[1].upper() in gene_to_sym:
            symbol = gene_to_sym[ls[1].upper()]
            try:
                numerical["Cancer_expression"][symbol] = [float(s) for s in ls[2:]]
            except ValueError:
                continue

In [236]:
#Dependency_map
#!wget https://ndownloader.figshare.com/files/31315996
#%mv 31315996 ./Repository/Feature/Crispr.csv
numerical["DepMap"]=newdic_num()
dfdep = pd.read_csv("./Repository/Feature/Crispr.csv")
dfdep = dfdep.drop("DepMap_ID",axis=1).dropna()
dfdep.columns = [g[:g.index("(")-1].upper() for g in dfdep.columns]
for gene in tqdm.notebook.tqdm(dfdep.columns):
    if gene.upper() in gene_to_sym:
        symbol = gene_to_sym[gene.upper()]
        numerical["DepMap"][symbol] = dfdep[gene].tolist()

  0%|          | 0/17386 [00:00<?, ?it/s]

In [1]:
#word2vec
import gensim
model = gensim.models.Word2Vec.load('./Reposiroty/word2vec_window10.model')
numerical["Word2Vec"] = newdic_num()
for symbol in set(gene_to_sym.values()):
    symbol = symbol.lower()
    if symbol in model.wv:
        numerical["Word2Vec"][symbol.upper()] = model.wv[symbol]

NameError: name 'newdic_num' is not defined

# STRING

In [287]:
#!wget http://version10.string-db.org/download/protein.links.v10/9606.protein.links.v10.txt.gz -O ./Repository/Feature/protein.links.v10.txt.gz
#!gunzip ./Repository/Feature/protein.links.v10.txt.gz

--2023-02-04 16:29:29--  http://version10.string-db.org/download/protein.links.v10/9606.protein.links.v10.txt.gz
version10.string-db.org (version10.string-db.org) をDNSに問いあわせています... 194.94.44.34
version10.string-db.org (version10.string-db.org)|194.94.44.34|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 46960139 (45M) [application/x-gzip]
`./Repository/Feature/protein.links.v10.txt.gz' に保存中

./Repository/Featur 100%[===================>]  44.78M   430KB/s    時間 1m 47s  

2023-02-04 16:31:16 (430 KB/s) - `./Repository/Feature/protein.links.v10.txt.gz' へ保存完了 [46960139/46960139]



In [10]:
#ID convert from Ensembl protein to HGNC
dfhg = pd.read_table("./data/mart_export.txt")
ensp_to_sym={}
for s,p in zip(dfhg["HGNC symbol"],dfhg["Protein stable ID"]):
    if type(s)!=str:
        continue
    s = s.upper()
    if s in gene_to_sym and p!="":
          ensp_to_sym[p]=gene_to_sym[s]

In [309]:
#!head ./Repository/Feature/protein.links.v10.txt -n 3
f2 = open("./Repository/feature/string.txt","w")
with open("./Repository/Feature/protein.links.v10.txt","r") as f:
    for line in f:
        ls = line.strip("\n").split(" ")
        if ls[0].startswith("9606."):
            protein_a = ls[0][5:]
            protein_b = ls[1][5:]
            if protein_a in ensp_to_sym and protein_b in ensp_to_sym:
                symbol_a = ensp_to_sym[protein_a]
                symbol_b = ensp_to_sym[protein_b]
                f2.write("\t".join([symbol_a,symbol_b,ls[2]])+"\n")
f2.close()

# Collection

In [260]:
with open("./Repository/feature/numerical_feature.txt","w") as f:
    for num in numerical:
        dic = numerical[num]
        for gene in dic:
            f.write(num+"\t"+gene+"\t")
            f.write(",".join([str(v) for v in dic[gene]]))
            f.write("\n")

In [318]:
import collections
def feature_filter(dic,n=5):
    l = []
    for value in dic.values():
        l += list(value)
    c = collections.Counter(l)
    ret = {}
    for key in dic.keys():
        ret[key] = {feature for feature in dic[key] if c[feature]>=n}
    return ret

In [319]:
#categorical features annotated on at least 10 genes were used 
with open("./Repository/feature/categorical_feature_10.txt","w") as f:
    for cat in categorical:
        dic = categorical[cat]
        dic = feature_filter(dic,n=10)
        for gene in dic:
            f.write(cat+"\t"+gene+"\t")
            f.write(",".join(dic[gene]))
            f.write("\n")